In [ ]:
# To begin, we initialize global variables, objects and datasets. 
# We also load the variouse packages required for constructing our examples:

# Import general purpose packages
import os
import json
import pandas as pd
import numpy as np
import datetime
import pytz
import warnings
import pprint

warnings.filterwarnings("ignore", module="matplotlib*")

import matplotlib.pyplot as plt

from datetime import datetime, timedelta
from pandas import json_normalize
from flatten_json import flatten

# Import lusid specific packages
import lusid
import lusid.models as models
from lusidtools.cocoon.cocoon import load_from_data_frame

from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken

# Set display configuration
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", lambda x: "%.5f" % x)
pd.set_option("display.max_rows", 3500)
pd.set_option("max_colwidth", 20)

# Use line magic function to enable matplotlib to work interactively with iPython
%matplotlib inline

# Set style to fivethirtyeight to create clean and clear looking graphs
plt.style.use("fivethirtyeight")

# Define a dictionary containing default plotting configurations
params = {
    "legend.fontsize": "small",
    "figure.figsize": (12, 4.5),
    "axes.labelsize": "small",
    "axes.titlesize": "medium",
    "xtick.labelsize": "small",
    "ytick.labelsize": "small",
}

plt.rcParams.update(params)

# Authenticate our user and create our API client
# secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
# api_factory = lusid.utilities.ApiClientFactory(
#     token=RefreshingToken(),
#     api_secrets_filename=secrets_path,
#     app_name="LusidJupyterNotebook",
# )

api_factory = ApiClientFactory(token=RefreshingToken())

api_status = pd.DataFrame(
    api_factory.build(lusid.ApplicationMetadataApi).get_lusid_versions().to_dict()
)

display(api_status)

# 1.2 Define Global Variables
## 1.2.1 Specify Notebook Scope and Start Date
The ```scope``` and ```start_date```will be used globally accross the Notebook.

In [ ]:
# Define a scope to hold data
scope = "ukIBOR"
start_date = "2021-01-01"

## 1.2.2 Define LUSID API's

In [ ]:
# Define the APIs we use
configuration_recipe_api = api_factory.build(lusid.ConfigurationRecipeApi)
reference_portfolios_api = api_factory.build(lusid.ReferencePortfolioApi)
aggregation_api = api_factory.build(lusid.AggregationApi)
instruments_api = api_factory.build(lusid.InstrumentsApi)
portfolios_api = api_factory.build(lusid.PortfoliosApi)
quotes_api = api_factory.build(lusid.QuotesApi)

# 2. Load Instrument Master
Our instrument master data will be loaded from two separate data sources, each used in a different reference portfolio. Each data source containing a different class of instrument:

The first containing UK equities, loaded form "data/benchmark/uk-stocks.csv"
The second containing UK fixed income, loaded from "data/benchmark/uk-bonds.csv"

## 2.1 Load the Equity Instruments

We begin by loading our equity instruments and mapping the accompanying identifiers and required fields to LUSID.

Our portfolio consists of a collection of various UK equities, weighted according to their proportion of the underlying portfolio. Each entry contains a "Name" attribute and a corresponding "Sector" property. We also have three identifiers for each equity, a "Ticker", "ISIN" and "SEDOL". Each of this fields can be mapped to a properties in LUSID. We be able to access each equity by it's "Ticker" value, which we will map to "ClientInternal" in LUSID.

## 2.1.1 Read Equity CSV File

In [ ]:
uk_stocks = pd.read_csv("data/benchmark/uk-stocks.csv")
uk_stocks_test = uk_stocks.head(3)

## 2.1.2 Load Equity Data to LUSID

In [ ]:
# Load the instruments into LUSID

instrument_identifier_mapping = {
    "Figi": "Figi",
    "Ticker": "Ticker",
    "Isin": "ISIN",
    "Sedol": "SEDOL",
    "Currency": "Currency",
}

instrument_mapping_required = {"name": "Name"}

instrument_mapping_optional = {}

responses = load_from_data_frame(
    api_factory=api_factory,
    # scope=scope,
    scope="ukIBOR",
    # scope="default",
    # mapping
    data_frame=uk_stocks,
    mapping_required=instrument_mapping_required,
    mapping_optional=instrument_mapping_optional,
    file_type="instrument",
    identifier_mapping=instrument_identifier_mapping,
    property_columns=["Sector"],
)

In [ ]:
definitions = {}

for index, row in uk_stocks.iterrows():
    definitions[row["Name"]] = models.InstrumentDefinition(
        name = row["Name"],
        identifiers = {
            "Figi": models.InstrumentIdValue(value = row["Figi"]),
            "Isin": models.InstrumentIdValue(value = row["ISIN"]),
            "Sedol": models.InstrumentIdValue(value = row["SEDOL"]),
            "Ticker": models.InstrumentIdValue(value = row["Ticker"])
        },
        definition = models.Equity(
            instrument_type = "Equity",
            dom_ccy = row["Currency"],
            identifiers = {}
        ))
    
upsert_instruments_response = instruments_api.upsert_instruments(
    request_body = definitions,
    scope = scope,
)

pprint.pprint(upsert_instruments_response)

## 2.2 Load the Fixed Income instruments
In a similar fashion, we continue by loading our fixed income data to LUSID. Our fixed income data comes with only a single identifying field - "ISIN" - as well as a "Name" and "Sector" property.

### 2.2.1 Read Fixed Income CSV File

In [ ]:
uk_bonds = pd.read_csv("data/benchmark/uk-bonds.csv")
uk_bonds.head(3)

### 2.2.2 Load Fixed Income Data to LUSID

In [ ]:
# Load the instruments into LUSID

instrument_identifier_mapping = {
    "Figi": "Figi",
    "Isin": "ISIN",
}

instrument_mapping_required = {"name": "Name"}

instrument_mapping_optional = {}

responses = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=uk_bonds,
    mapping_required=instrument_mapping_required,
    mapping_optional=instrument_mapping_optional,
    file_type="instrument",
    identifier_mapping=instrument_identifier_mapping,
    property_columns=["Sector"],
)

In [ ]:
definitions = {}

for index, row in uk_bonds.iterrows():
    definitions[row["Name"]] = models.InstrumentDefinition(
        name = row["Name"],
        identifiers = {
            "Figi": models.InstrumentIdValue(value = row["Figi"]),
            "Isin": models.InstrumentIdValue(value = row["ISIN"])
        },
        definition = models.Bond(
            start_date = "2024-05-22T00:00:00.0000000+00:00",
            maturity_date="2026-10-22T00:00:00.0000000+00:00",
            instrument_type = "Bond",
            dom_ccy = row["Currency"],
            flow_conventions = models.FlowConventions(
                currency = row["Currency"],
                payment_frequency = "1M",
                day_count_convention = "Actual360",
                roll_convention = "1",
                payment_calendars = ["GBP"],
                reset_calendars = []
            ),
            principal = 100,
            coupon_rate = 0.015,
            identifiers = {}
        ))
    
upsert_instruments_response = instruments_api.upsert_instruments(
    request_body = definitions,
    scope = scope,
)

pprint.pprint(upsert_instruments_response)

## Load Instrument Data into Lusid

# 3. Create reference portfolios
Next, we construct our reference portfolios. These include:

Our UK Equity portfolio, consisting of various weights of UK equities
Our UK Fixed Income portfolio, consisting of variouse weights of UK Bonds
And our Blended Portfolio, which constitutes some portion of our Equity and Fixed Income portfolios
## 3.1 Define Names for Reference Portfolios

In [ ]:
uk_bond_index = "uKBondIndex"
uk_equity_index = "uKEquityIndex"
uk_blended_index = "ukBlendedIndex"

# Define our list of Reference portfolios
reference_portfolios = [uk_bond_index, uk_equity_index, uk_blended_index]

## 3.2 Create Reference Portfolios
For each reference portfolio create an instance of the portfolio in LUSID. Specify the scope and time created to be the values specified globally in our Notebook. A reference portfolio in LUSID allows the storage of a portfolio which contains a set of constituent instruments and weights which can change over time.

For additional information on Reference Portfolios, please see the following KB article.

NOTE: The time when a portfolio is created must occur before or at the point when that any of it's constituents are upserted

In [ ]:
# The `create_reference_portfolio()` method implements LUSID API `CreateReferencePortfolio` method. 
# More information on this API method can be found in the following [LUSID API Docs](https://www.lusid.com/docs/api/#tag/Reference-Portfolio)

for portfolio in reference_portfolios:

    try:

        response = reference_portfolios_api.create_reference_portfolio(
            scope=scope,
            create_reference_portfolio_request=models.CreateReferencePortfolioRequest(
                display_name=portfolio, 
                code=portfolio, 
                created="2010-01-01", 
                instrument_scopes=[scope],
                base_currency="GBP",
            ),
        )

    except lusid.ApiException as e:
        print(json.loads(e.body)["title"])

# 4. Load constituents
## 4.1 Securitise the reference portfolios
We now securitise each of the reference portfolios, allowing us to generate a blended benchmark containing multiple reference portfolios reperesented in LUSID. The reference portfolio can be configured to automatically float the weights of the underlying indices (the securitised reference portfolio constituents) so that they move in line with the movements of the index.

For additional information on the Securitisation of Reference Portfolios, please see the following KB article

In [ ]:
response = instruments_api.upsert_instruments(
    scope=scope,
    request_body={
        f"upsert_instrument_{portfolio}": models.InstrumentDefinition(
            name=portfolio,
            identifiers={
                "ClientInternal": models.InstrumentIdValue(value=f"inst_{portfolio}"),
            },
            look_through_portfolio_id=models.ResourceId(
                scope=scope, 
                code=portfolio,
            ),
        ) for portfolio in reference_portfolios
    }
)

## 4.2 Load constituents for FI and EQ reference portfolios
We now create Index constituents for the time period in consideration for each of the reference portfolios.

These constituents should not overlap (i.e. be on the same instrument) with any existing constituents.

In [ ]:
# Add uk bond index constituents
constituents = [
    models.ReferencePortfolioConstituentRequest(
        instrument_identifiers={
            "Instrument/default/Figi": row["Figi"]
        },
        weight=row["Weighting"],
        currency="GBP",
    ) for _, row in uk_bonds.iterrows()
]

# Create our request to add our constituents
constituents_request = models.UpsertReferencePortfolioConstituentsRequest(
    effective_from=start_date,
    weight_type="Periodical",
    period_type="Quarterly",
    period_count=1,
    constituents=constituents,
)

# Call LUSID to upsert our constituents into our reference portfolio
response = reference_portfolios_api.upsert_reference_portfolio_constituents(
    scope=scope,
    code=uk_bond_index,
    upsert_reference_portfolio_constituents_request=constituents_request,
)

print(f"Constituents Upserted for {uk_bond_index}")

In [ ]:
# Add uk equity index constituents
constituents = [
    models.ReferencePortfolioConstituentRequest(
        instrument_identifiers={
            "Instrument/default/Figi": row["Figi"]
        },
        weight=row["Weighting"],
        currency="GBP",
    ) for _, row in uk_stocks.iterrows()
]

# Create our request to add our constituents
constituents_request = models.UpsertReferencePortfolioConstituentsRequest(
    effective_from=start_date,
    weight_type="Periodical",
    period_type="Quarterly",
    period_count=1,
    constituents=constituents,
)

# Call LUSID to upsert our constituents into our reference portfolio
response = reference_portfolios_api.upsert_reference_portfolio_constituents(
    scope=scope,
    code=uk_equity_index,
    upsert_reference_portfolio_constituents_request=constituents_request,
)

print(f"Constituents Upserted for {uk_equity_index}")

## 4.3 Load constituents for blended benchmark
Now, we create a set of constituents for our blended benchmark reference portfolio. The blended portfolio is made up by 60% of the UK bond portfolio and 40% of the UK equity portfolio. So we wish to re distribute our portfolio holding to meet the specified weighing values. We also model the constituent rule to be applied periodically on a quaterly bases.

In [ ]:
weightings = [(uk_bond_index, 0.6), (uk_equity_index, 0.4)]

In [ ]:
# We specify quarterly rebalancing of the blended benchmark indexby specifying `weight_type` and `period_type` as shown. 
# The period count specifies how many `period_type` duration must pass before portfolio is rebalanced. 
# Additional details on reference portfolio constituents can be found in the following [KB Article](https://support.lusid.com/knowledgebase/article/KA-01852/en-us)

# create reference portfolio constituents
constituents = [
    models.ReferencePortfolioConstituentRequest(
        instrument_identifiers={"Instrument/default/ClientInternal": f"inst_{port}"},
        weight=weighting,
        currency="GBP",
    ) for port, weighting in weightings
]

# Create our request to add our constituents
constituents_request = models.UpsertReferencePortfolioConstituentsRequest(
    effective_from=start_date,
    weight_type="Periodical",
    period_type="Quarterly",
    period_count=1,
    constituents=constituents,
)

# Call LUSID to upsert our constituents into our reference portfolio
response = reference_portfolios_api.upsert_reference_portfolio_constituents(
    scope=scope,
    code=uk_blended_index,
    upsert_reference_portfolio_constituents_request=constituents_request,
)

print(f"Constituents Upserted for {weightings[0][0]} and {weightings[1][0]}")

# 5. Upsert quotes
We now load in the relevant market prices from 1st of Jan to 1st of Apr 2021.

In [ ]:
# instruments_ids = [uk_bond_index, uk_equity_index, uk_blended_index]
instruments_ids = [f"inst_{code}" for code in reference_portfolios]

# Use dictionary comprehension to generate dictionary of Luids
luids = {
    instrument_id: instruments_api.get_instrument(
        identifier_type="ClientInternal",
        identifier=instrument_id,
        scope=scope,
    ).lusid_instrument_id
    for instrument_id in instruments_ids
}

In [ ]:
instrument_data = {
    1: {"date": "2021-01-01", "instrument": uk_bond_index, "price": 100},
    2: {"date": "2021-01-31", "instrument": uk_bond_index, "price": 130},
    3: {"date": "2021-03-31", "instrument": uk_bond_index, "price": 129},
    4: {"date": "2021-04-01", "instrument": uk_bond_index, "price": 136},
    5: {"date": "2021-01-01", "instrument": uk_equity_index, "price": 100},
    6: {"date": "2021-01-31", "instrument": uk_equity_index, "price": 80},
    7: {"date": "2021-03-31", "instrument": uk_equity_index, "price": 90},
    8: {"date": "2021-04-01", "instrument": uk_equity_index, "price": 86},
}

# Create quote requests
instrument_quotes = {
    f"upsert_request_{key}": models.UpsertQuoteRequest(
        quote_id=models.QuoteId(
            quote_series_id=models.QuoteSeriesId(
                provider="Lusid",
                instrument_id=luids[f'inst_{data["instrument"]}'],
                instrument_id_type="LusidInstrumentId",
                quote_type="Price",
                field="mid",
            ),
            effective_at=data["date"],
        ),
        metric_value=models.MetricValue(value=data["price"], unit="GBP"),
    ) for key, data in instrument_data.items()
}

# Upsert the quotes into LUSID
response = quotes_api.upsert_quotes(scope=scope, request_body=instrument_quotes)

# 6. Get blended index floating weight over timeline
In this section, we track the benchmark weights across a Q1 timeline, with the floating weights resetting on 1 April.

## 6.1 Get weights for 1 Jan
On the 1 Jan, which is Day 1 for the portfolio, the fixed weight is equal to the floating weight

In [ ]:
get_constituents = reference_portfolios_api.get_reference_portfolio_constituents(
    scope=scope,
    code=uk_blended_index,
    effective_at="2021-01-01",
)

pd.DataFrame([flatten(item.to_dict()) for item in get_constituents.constituents])

## 6.2 Get weights for 31 Jan
The floating weights have moved as the underlying indicies have moved
The bond portfolio has performed well, meaning the float weight has jumpted from 60% to ~ 70%
The equity portfolio has not performed as well, meaning its floating weight has dropped to ~30%

In [ ]:
get_constituents = reference_portfolios_api.get_reference_portfolio_constituents(
    scope=scope, 
    code=uk_blended_index, 
    effective_at="2021-01-31",
)

pd.DataFrame([flatten(item.to_dict()) for item in get_constituents.constituents])

## 6.3 Get weights for 31 March
The floating weights have continued to move

In [ ]:
get_constituents = reference_portfolios_api.get_reference_portfolio_constituents(
    scope=scope, 
    code=uk_blended_index, 
    effective_at="2021-03-31",
)

pd.DataFrame([flatten(item.to_dict()) for item in get_constituents.constituents])

## 6.4 Weights are reset on 1 April
On the 1 April, the weights are reset, and the weight and floating rate are equal. This is driven by the period type which we set to "Quarterly" in LUSID.

In [ ]:
get_constituents = reference_portfolios_api.get_reference_portfolio_constituents(
    scope=scope, 
    code=uk_blended_index, 
    effective_at="2021-04-01",
)

pd.DataFrame([flatten(item.to_dict()) for item in get_constituents.constituents])

# Produce Look-through recipe

In [ ]:
# Create look-through-enabled recipe
lookthrough_config_recipe = models.ConfigurationRecipe(
    scope=scope,
    code="lookthrough",
    market=models.MarketContext(
        market_rules=[
            models.MarketDataKeyRule(
                key="Quote.*.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Price",
                field="mid",
            ),
            models.MarketDataKeyRule(
                key="FX.CurrencyPair.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Rate",
                field="mid",
            ),
        ],
        options=models.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="ClientInternal",
            default_scope=scope,
            attempt_to_infer_missing_fx=True,
        ),
    ),
    pricing=models.PricingContext(
        options={"AllowPartiallySuccessfulEvaluation": False},
        # toggle look through
        model_rules=[
            models.VendorModelRule(
                supplier="Lusid",
                model_name="SimpleStatic",
                instrument_type="SimpleInstrument",
                model_options=models.IndexModelOptions(portfolio_scaling="Sum", model_options_type="IndexModelOptions"),
            )
        ],
    ),
)

upsert_configuration_recipe_response = (
    configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=models.UpsertRecipeRequest(
            configuration_recipe=lookthrough_config_recipe
        )
    )
)

# Get Valuation

In [ ]:
recipe_scope = "ukIBOR"
recipe_code = "lookthrough"

portfolio_scope = "ukIBOR"

# portfolio_code = uk_bond_index
# portfolio_code = uk_equity_index
portfolio_code = uk_blended_index

valuation_request = models.ValuationRequest(
    recipe_id={"scope":recipe_scope,"code":recipe_code},
    metrics=[{"key":"Instrument/default/Name","op":"Value","options":{}},
             {"key":"Instrument/default/LusidInstrumentId","op":"Value","options":{}},
             {"key":"Valuation/PV","op":"Value","options":{}},
             {"key":"Lookthrough/LookthroughWeight","op":"Value","options":{}}
             # {"key":"Aggregation/GrossExposureWeight","op":"Value","options":{}},
             # {"key":"Aggregation/NetExposureWeight","op":"Value","options":{}}
            ],
    group_by=["Instrument/default/Name"],
    report_currency="GBP",
    equip_with_subtotals=False,
    return_result_as_expanded_types=False,
    portfolio_entity_ids=[{"scope":portfolio_scope,"code":portfolio_code,"portfolioEntityType":"SinglePortfolio"}],
    valuation_schedule={
                          "effectiveAt":"2024-05-22T00:00:00.0000000+00:00",
                          "tenor":"1D","rollConvention":"None",
                          "holidayCalendars":[],
                          "valuationDateTimes":[],
                          "businessDayConvention":"F"})

try:
    # GetValuation: Perform valuation for a list of portfolios and/or portfolio groups
    api_response = aggregation_api.get_valuation(valuation_request=valuation_request)
    pprint.pprint(api_response)
except Exception as e:
    print("Exception when calling AggregationApi->get_valuation: %s\n" % e)
